# Working with JWST data for deconvolution training
This is a tutorial/walkthrough for the research that I did with COSMOS-Web's data release. This is intended to be used to grab images from a public data set, given a pair of coordinates for a specific galaxy.

## Working with one .fits file

In [5]:
import matplotlib.pyplot as plt
import matplotlib
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
fitspath = 'fits/'

##### Pixel Input

In [ ]:
obj = pd.read_csv('testinput.txt',sep=',',header=0)
example = fits.open(fitspath+'example.fits.gz')
# load in image
image = example[1].data
header = example[1].header
w = WCS(header)

In [ ]:
for i in range(len(obj)): # for each object
    # get image bounds for stamp
    left = int((obj['xcenter'][i]-(obj['size'][i]/2)))
    right = int((obj['xcenter'][i]+(obj['size'][i]/2)))
    down = int((obj['ycenter'][i]-(obj['size'][i]/2)))
    up = int((obj['ycenter'][i]+(obj['size'][i]/2)))
    # crop image with matplotlib
    plt.imshow(image[down:up,left:right],cmap='gray',vmin=0,vmax=1.5)
    plt.title(f'f770w_{obj['ID'][i]}')
    plt.savefig(f'output/f770w_{obj['ID'][i]}.png')


##### RA DEC Input

In [ ]:
obj = pd.read_csv('radecinput.txt',sep=',',header=0)
example = fits.open(fitspath+'example.fits.gz')
# load in image
image = example[1].data
header = example[1].header
w = WCS(header)

In [ ]:
ra, dec, asecr = 149.9971059,2.0278320,2.367
xpix, ypix = w.wcs_world2pix(ra,dec,1)
xpix = round(float(xpix))
ypix = round(float(ypix))
rpix = round(float((asecr/3600)/header['CDELT1'])) 
print(xpix,ypix,rpix)

Save as .PNG files

In [ ]:
for i in range(len(obj)): # for each object
    xpix, ypix = w.wcs_world2pix(obj['RA'][i],obj['DEC'][i],1) # convert from RA/DEC to pixel position
    xpix = round(float(xpix))
    ypix = round(float(ypix))
    rpix = round(float((obj['asecr'][i]/3600)/header['CDELT1'])) # arcsec / 3600 = deg / (pix/deg) = pix radius
    #print(xpix,ypix,rpix) # debug print file values
    # get image bounds for stamp
    left = int((xpix-rpix))
    right = int((xpix+rpix))
    down = int((ypix-rpix))
    up = int((ypix+rpix))
    #print(left,right,up,down) # debug print image bounds
    # crop image with matplotlib
    plt.imshow(image[down:up,left:right],cmap='gray',vmin=0,vmax=1.5,origin='lower')
    plt.title(f'f770w_{obj['ID'][i]}')
    plt.savefig(f'output/f770w_{obj['ID'][i]}.png')


Save as .fits files

In [25]:
for i in range(len(obj)): # for each object
    xpix, ypix = w.wcs_world2pix(obj['RA'][i],obj['DEC'][i],1) # convert from RA/DEC to pixel position
    xpix = round(float(xpix))
    ypix = round(float(ypix))
    rpix = round(float((obj['asecr'][i]/3600)/header['CDELT1'])) # arcsec / 3600 = deg / (pix/deg) = pix radius
    #print(xpix,ypix,rpix) # debug print file values
    # get image bounds for stamp
    left = int((xpix-rpix))
    right = int((xpix+rpix))
    down = int((ypix-rpix))
    up = int((ypix+rpix))
    #print(left,right,up,down) # debug print image bounds
    # fits file saving
    outfile = f'output/f770w_{obj['ID'][i]}.fits' # prepare outbound filename
    prihdu = fits.PrimaryHDU() # set up primary first HDU
    prihdu.data = example[0].data # carry over data from original HDU
    prihdu.header = example[0].header # carry over header from original HDU
    prihdu.header.update(w[down:up,left:right].to_header()) # crop original data in HDU to fit our new image range
    scihdu = fits.ImageHDU() # set up sci HDU
    scihdu.data = example[1].data[down:up,left:right] # carry over data from original image, cropped to our bounds
    scihdu.header = example[1].header # carry over data from original sci header
    scihdu.header.update(w[down:up,left:right].to_header()) # crop data in sci header to our new bounds using WCS function and bounds
    hdulist = fits.HDUList([prihdu, scihdu]) # put both hdu's in hdulist
    hdulist.writeto(outfile, overwrite=True) # save hdu's in new fits file


Post-fits-check

In [ ]:
test = fits.open('output/f770w_CoolGal7.fits')
test.info()
#test[0].header
#test[1].header
imagetest = test[1].data
# Is there an image on the Sci HDU? Was it cropped?
#plt.imshow(imagetest,cmap='gray',vmin=0,vmax=1.5,origin='lower')
#plt.title(f'f770w_CoolGal7')

Save as tiles

In [ ]:
num_images  = 9
num_cols    = min(num_images, 3)
num_rows    = int(num_images / 3) + (1 if num_images % num_cols != 0 else 0)

# Create a grid of subplots.
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12,12))
list_axes = list(axes.flat)
for i in range(0,num_images): # for each object
    xpix, ypix = w.wcs_world2pix(obj['RA'][i],obj['DEC'][i],1) # convert from RA/DEC to pixel position
    xpix = round(float(xpix))
    ypix = round(float(ypix))
    rpix = round(float((obj['asecr'][i]/3600)/header['CDELT1'])) # arcsec / 3600 = deg / (pix/deg) = pix radius
    #print(xpix,ypix,rpix) # debug print file values
    # get image bounds for stamp
    left = int((xpix-rpix))
    right = int((xpix+rpix))
    down = int((ypix-rpix))
    up = int((ypix+rpix))
    #print(left,right,up,down) # debug print image bounds
    # crop image with matplotlib
    list_axes[i].imshow(image[down:up,left:right],cmap='gray',vmin=0,vmax=1.5,origin='lower')
    list_axes[i].set_title(f'f770w_{obj['ID'][i]}')
plt.tight_layout()
plt.savefig('output/galcollage.png',dpi=150)
plt.show()

## Iterating through .fits

Logic:
* load in os
* get dependencies
* have input ra/dec to check for
* for every fits file in directory, </br>
    if testra and testdec are within the ranges from the header</br>
    &ensp;do the imaging stuff </br>
    else </br>
    &ensp;don't do that and check the next one </br>

In [1]:
from os import listdir
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
path = r"/home/eliasw/OneDrive/Research/Deconvolution/fits"

In [ ]:
for file in listdir(path):
    print(file)

In [1]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np

In [ ]:
ra = []
dec = []
for i in range(len(obj)):
    print(f'Looking for {obj['Name'][i]}..') # Debug: What object are we on?
    for file in listdir(path): # look through each file
        f = fits.open(path + "/" + file)
        image = f[1].data
        w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
        ramax,decmax = max(w.all_pix2world((image),1)) # takes the ra from the leftmost pixel value (most ra) [0] and the dec from the top edge (most dec) [given by header axis length]
        print(ramax,decmax)
        if ((ramin<=obj['RA'][i]<=ramax) & (decmin<=obj['DEC'][i]<=decmax)): # if our object's values are between the files values
            # print('Here!')
            print(f'Object is in {file}')
            f.close() # close up shop
            break # if we find it, we can leave.
        else:
            # print('Absent.') # it aint here
            f.close() # close up shop

In [ ]:
# targets we are looking for
ra = 150.3587481
dec = 2.0068304
for file in listdir(path): # look through each file
    print(file) # debug: print file name
    header = fits.getheader(path + "/" + file, ext=1)
    w = WCS(header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
    ramin,decmin = w.pixel_to_world_values(int(header['NAXIS1']),0) # takes the ra from the rightmost pixel value (least ra) [given by header axis length] and the dec from the bottom edge (bottom dec) [0]
    ramax,decmax = w.pixel_to_world_values(0,int(header['NAXIS2'])) # takes the ra from the leftmost pixel value (most ra) [0] and the dec from the top edge (most dec) [given by header axis length]
    print('RA (min/max)',ramin,ramax) # debug: show RA min/max
    print('DEC (min/max)',decmin,decmax) # debug: show DEC min/max
    if (ramin<=ra<=ramax) & (decmin<=dec<=decmax): # if our values are between the files values
        print('Here!')
        break # if we find it, we can leave.
    else:
        print('Absent.') # it aint here

In [ ]:
# targets we are looking for
ra = 150.3587481
dec = 2.0068304
for file in listdir(path): # look through each file
    print(file) # debug: print file name
    header = fits.getheader(path + "/" + file, ext=1)
    w = WCS(header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
    ramin,decmin = w.pixel_to_world_values(int(header['NAXIS1']),0) # takes the ra from the rightmost pixel value (least ra) [given by header axis length] and the dec from the bottom edge (bottom dec) [0]
    ramax,decmax = w.pixel_to_world_values(0,int(header['NAXIS2'])) # takes the ra from the leftmost pixel value (most ra) [0] and the dec from the top edge (most dec) [given by header axis length]
    print('RA (min/max)',ramin,ramax) # debug: show RA min/max
    print('DEC (min/max)',decmin,decmax) # debug: show DEC min/max
    if (ramin<=ra<=ramax) & (decmin<=dec<=decmax): # if our values are between the files values
        print('Here!')
        break # if we find it, we can leave.
    else:
        print('Absent.') # it aint here

In [ ]:
# targets we are looking for
ra = 150.3587481
dec = 2.0068304
for file in listdir(path): # look through each file
    print(file) # debug: print file name
    header = fits.getheader(path + "/" + file, ext=1)
    w = WCS(header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
    ramin,decmin = w.pixel_to_world_values(int(header['NAXIS1']),0) # takes the ra from the rightmost pixel value (least ra) [given by header axis length] and the dec from the bottom edge (bottom dec) [0]
    ramax,decmax = w.pixel_to_world_values(0,int(header['NAXIS2'])) # takes the ra from the leftmost pixel value (most ra) [0] and the dec from the top edge (most dec) [given by header axis length]
    # print(ramin,ramax) # debug: show RA min/max
    # print(decmin,decmax) # debug: show DEC min/max
    if (ramin<=ra<=ramax) & (decmin<=dec<=decmax): # if our values are between the files values
        print('Here!')
        print('Processing image... Please wait dude..') # Debug: Progress Bar
        image = fits.getdata(path + "/" + file)
        xpix, ypix = w.wcs_world2pix(ra,dec,1) # convert from RA/DEC to pixel position
        xpix = round(float(xpix))
        ypix = round(float(ypix))
        rpix = round(100) # arcsec / 3600 = deg / (pix/deg) = pix radius
        # print(xpix,ypix,rpix) # debug: print file values
        # crop image with matplotlib
        plt.imshow(image[int((ypix-rpix)):int((ypix+rpix)),int((xpix-rpix)):int((xpix+rpix))],cmap='gray',vmin=0,vmax=1.5,origin='lower')
        plt.title(f'f770w_objTEST')
        plt.savefig(f'output/objTEST.png')
        break # if we find it, we can leave.
    else:
        print('Absent.') # it aint here

Can we make opening .fits more efficent?

In [ ]:
# targets we are looking for
ra = 150.3587481
dec = 2.0068304
for file in listdir(path): # look through each file
    print(file) # debug: print file name
    f = fits.open(path + "/" + file)
    w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
    ramin,decmin = w.pixel_to_world_values(int(f[1].header['NAXIS1']),0) # takes the ra from the rightmost pixel value (least ra) [given by header axis length] and the dec from the bottom edge (bottom dec) [0]
    ramax,decmax = w.pixel_to_world_values(0,int(f[1].header['NAXIS2'])) # takes the ra from the leftmost pixel value (most ra) [0] and the dec from the top edge (most dec) [given by header axis length]
    # print(ramin,ramax) # debug: show RA min/max
    # print(decmin,decmax) # debug: show DEC min/max
    if (ramin<=ra<=ramax) & (decmin<=dec<=decmax): # if our values are between the files values
        print('Here!')
        print('Processing image... Please wait dude..') # Debug: Progress Bar
        image = f[1].data
        xpix, ypix = w.wcs_world2pix(ra,dec,1) # convert from RA/DEC to pixel position
        xpix = round(float(xpix))
        ypix = round(float(ypix))
        rpix = round(100) # deg / (pix/deg) = pix radius
        # print(xpix,ypix,rpix) # debug: print file values
        # crop image with matplotlib
        plt.imshow(image[int((ypix-rpix)):int((ypix+rpix)),int((xpix-rpix)):int((xpix+rpix))],cmap='gray',vmin=0,vmax=1.5,origin='lower')
        plt.title(f'f770w_objTEST')
        plt.savefig(f'output/objTEST.png')
        f.close()
        break # if we find it, we can leave.
    else:
        print('Absent.') # it aint here
        f.close()

YES!

Time for more inputs! Iteration!

In [6]:
# these are the droi- targets we are looking for
obj = pd.read_csv('multiinput.txt',sep='\t',header=0)

In [ ]:
obj

In [ ]:
for i in range(len(obj)):
    print(f'Looking for {obj['Name'][i]}..') # Debug: What object are we on?
    for file in listdir(path): # look through each file
        # print(file) # debug: print file name
        f = fits.open(path + "/" + file)
        w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
        # TROUBLE HERE V V V
        print('len=',len((0,f[1].header['NAXIS2'])))
        print(f[1].header['NAXIS1'])
        print(f[1].header['NAXIS2'])
        ymax=
        ramin,decmin = w.all_pix2world((f[1].header['NAXIS1'],0)) # takes the ra from the rightmost pixel value (least ra) [given by header axis length] and the dec from the bottom edge (bottom dec) [0]
        ramax,decmax = w.all_pix2world((0,f[1].header['NAXIS2'])) # takes the ra from the leftmost pixel value (most ra) [0] and the dec from the top edge (most dec) [given by header axis length]
        # print(ramin,ramax) # debug: show RA min/max
        # print(decmin,decmax) # debug: show DEC min/max
        if ((ramin<=obj['RA'][i]<=ramax) & (decmin<=obj['DEC'][i]<=decmax)): # if our object's values are between the files values
            # print('Here!')
            print(f'Object is in {file}')
            # print('Processing image... Please wait dude..') # Debug: Progress Bar
            image = f[1].data
            xpix, ypix = w.wcs_world2pix(obj['RA'][i],obj['DEC'][i],1) # convert from RA/DEC to pixel position
            xpix = round(float(xpix))
            ypix = round(float(ypix))
            rpix = round(obj['rdeg'][i]/header['CDELT1']) # deg / (pix/deg) = pix radius
            # print(xpix,ypix,rpix) # debug: print file values
            # crop image with matplotlib
            plt.imshow(image[int((ypix-rpix)):int((ypix+rpix)),int((xpix-rpix)):int((xpix+rpix))],cmap='gray',vmin=0,vmax=1.5,origin='lower')
            plt.title(f'f770w_obj{obj['Name'][i]}')
            plt.savefig(f'output/obj{obj['Name'][i]}.png')
            f.close() # close up shop
            break # if we find it, we can leave.
        else:
            # print('Absent.') # it aint here
            f.close() # close up shop

And its broken!!

We need to make a better way to find the range of coordinates

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp

In [ ]:
# Open the FITS file and get WCS information
fits_file = 'fits/mosaic_miri_f770w_COSMOS-Web_60mas_A4_v0_5_i2d.fits.gz'
hdul = fits.open(fits_file)
wcs = WCS(hdul[1].header)
image_shape = hdul[1].data.shape  # Assuming 2D image

# Function for parallel RA/Dec conversion
def convert_pixels_to_radec(pixel_chunk, wcs):
    return wcs.all_pix2world(pixel_chunk, 1)
# Generate a grid of pixel coordinates for the entire image
y_pixels, x_pixels = np.indices(image_shape)

# Define subsampling factors for different regions
edge_subsample_factor = 10   # Dense sampling for edge
middle_subsample_factor = 1000  # Sparse sampling for the middle

# Define the boundaries for edges and middle regions
# Shrink the edge region, so more of the image is middle
edge_boundary = (image_shape[0] // 10, image_shape[1] // 10)  # Shrinking the edge (12.5% from edges)
middle_boundary = (image_shape[0] // 4, image_shape[1] // 4)  # Expanding the middle region (25% from edges)

# Generate a grid of pixel coordinates for the entire image
y_pixels, x_pixels = np.indices(image_shape)

# Sample the edges (outside the shrunk middle region)
x_pixels_edges = np.concatenate([x_pixels[::edge_subsample_factor, 0],  # Left edge
                                 x_pixels[::edge_subsample_factor, -1],  # Right edge
                                 x_pixels[0, ::edge_subsample_factor],  # Top edge
                                 x_pixels[-1, ::edge_subsample_factor]])  # Bottom edge
y_pixels_edges = np.concatenate([y_pixels[::edge_subsample_factor, 0], 
                                 y_pixels[::edge_subsample_factor, -1],
                                 y_pixels[0, ::edge_subsample_factor],
                                 y_pixels[-1, ::edge_subsample_factor]])

# Sample the middle region (less dense than center but more dense than edges)
x_pixels_middle = x_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                           edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]
y_pixels_middle = y_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                           edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]


# Combine all sampled pixels
pixels_combined = np.column_stack([np.concatenate([x_pixels_edges.ravel(), x_pixels_middle.ravel()]),
                                   np.concatenate([y_pixels_edges.ravel(), y_pixels_middle.ravel()])])

# Split the pixel blocks into chunks for parallel processing
num_cores = mp.cpu_count()
pixel_chunks = np.array_split(pixels_combined, num_cores)

# Convert pixel chunks to RA/Dec in parallel
with mp.Pool(num_cores) as pool:
    ra_dec_chunks = pool.starmap(convert_pixels_to_radec, [(chunk, wcs) for chunk in pixel_chunks])

# Combine the RA/Dec results
ra_dec = np.concatenate(ra_dec_chunks, axis=0)

# Extract Right Acension (RA) values and find the min and max
ra_values = ra_dec[:, 0]  # RA is the first column (in degrees)
ra_min = np.min(ra_values)
ra_max = np.max(ra_values)

print(f"RA range: {ra_min} to {ra_max} degrees")

# Extract Declination (Dec) values and find the min and max
dec_values = ra_dec[:, 1]  # Dec is the second column (in degrees)
dec_min = np.min(dec_values)
dec_max = np.max(dec_values)

print(f"Dec range: {dec_min} to {dec_max} degrees")

Time to implement

Final?

In [37]:
from os import listdir
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
import numpy as np
import multiprocessing as mp
path = r"/home/eliasw/OneDrive/Research/Deconvolution/fits"

import warnings
warnings.filterwarnings("ignore")

Make some functions

In [2]:
# Function for parallel RA/Dec conversion
def convert_pixels_to_radec(pixel_chunk, wcs):
    return w.all_pix2world(pixel_chunk, 1, output_verify='silentfix')

In [4]:
def bounds(image,w):
    image_shape = image.shape  # Assuming 2D image

    # Generate a grid of pixel coordinates for the entire image
    y_pixels, x_pixels = np.indices(image_shape)

    # Define subsampling factors for different regions
    edge_subsample_factor = 10   # Dense sampling for edge
    middle_subsample_factor = 1000  # Sparse sampling for the middle

    # Define the boundaries for edges and middle regions
    # Shrink the edge region, so more of the image is middle
    edge_boundary = (image_shape[0] // 10, image_shape[1] // 10)  # Shrinking the edge (12.5% from edges)
    middle_boundary = (image_shape[0] // 4, image_shape[1] // 4)  # Expanding the middle region (25% from edges)

    # Generate a grid of pixel coordinates for the entire image
    y_pixels, x_pixels = np.indices(image_shape)

    # Sample the edges (outside the shrunk middle region)
    x_pixels_edges = np.concatenate([x_pixels[::edge_subsample_factor, 0],  # Left edge
                                    x_pixels[::edge_subsample_factor, -1],  # Right edge
                                    x_pixels[0, ::edge_subsample_factor],  # Top edge
                                    x_pixels[-1, ::edge_subsample_factor]])  # Bottom edge
    y_pixels_edges = np.concatenate([y_pixels[::edge_subsample_factor, 0], 
                                    y_pixels[::edge_subsample_factor, -1],
                                    y_pixels[0, ::edge_subsample_factor],
                                    y_pixels[-1, ::edge_subsample_factor]])

    # Sample the middle region (less dense than center but more dense than edges)
    x_pixels_middle = x_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                            edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]
    y_pixels_middle = y_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                            edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]


    # Combine all sampled pixels
    pixels_combined = np.column_stack([np.concatenate([x_pixels_edges.ravel(), x_pixels_middle.ravel()]),
                                    np.concatenate([y_pixels_edges.ravel(), y_pixels_middle.ravel()])])

    # Split the pixel blocks into chunks for parallel processing
    num_cores = mp.cpu_count()
    pixel_chunks = np.array_split(pixels_combined, num_cores)

    # Convert pixel chunks to RA/Dec in parallel
    with mp.Pool(num_cores) as pool:
        ra_dec_chunks = pool.starmap(convert_pixels_to_radec, [(chunk, w) for chunk in pixel_chunks])

    # Combine the RA/Dec results
    ra_dec = np.concatenate(ra_dec_chunks, axis=0)

    # Extract Right Acension (RA) values and find the min and max
    ra_values = ra_dec[:, 0]  # RA is the first column (in degrees)
    ramin = np.min(ra_values)
    ramax = np.max(ra_values)

    # Extract Declination (Dec) values and find the min and max
    dec_values = ra_dec[:, 1]  # Dec is the second column (in degrees)
    decmin = np.min(dec_values)
    decmax = np.max(dec_values)

    return ramin,ramax,decmin,decmax

In [5]:
# these are the droi- targets we are looking for
obj = pd.read_csv('multiinput.txt',sep='\t',header=0)

In [6]:
obj

,Name,RA,DEC,rdeg
0,Big Guy,150.424830,2.066517,0.002781
1,Number 6,150.165187,2.062385,0.000905
2,Long One,149.866885,2.051756,0.000998
3,IDK,149.876102,2.072738,0.000467
4,Another Big,150.279300,1.904663,0.003116
5,Spiral Dude,149.996595,2.119639,0.000964
6,Galaxy,150.360678,1.988029,0.001422
7,Number 17,150.178078,1.935832,0.000625


In [ ]:
for i in range(len(obj)):
    print(f'Looking for {obj['Name'][i]}..') # Debug: What object are we on?
    for file in listdir(path): # look through each file
        print(file) # debug: print file name
        f = fits.open(path + "/" + file)
        w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
        image = f[1].data
        # get coord bounds
        ramin,ramax,decmin,decmax = bounds(image,w)
        xpix, ypix = w.all_world2pix(obj['RA'][i],obj['DEC'][i],1) # convert from RA/DEC to pixel position
        if (0 <= xpix <= f[1].header['NAXIS1'] and 0 <= ypix <= f[1].header['NAXIS2']):
            if ((ramin<=obj['RA'][i]<=ramax) & (decmin<=obj['DEC'][i]<=decmax)) & (f[4].data[int(ypix),int(xpix)] != 0): # if our object's values are between the files values
                print(f'Object is in {file}')
                # print('Processing image... Please wait dude..') # Debug: Progress Bar
                xpix = round(float(xpix))
                ypix = round(float(ypix))
                rpix = round(obj['rdeg'][i]/f[1].header['CDELT1']) # deg / (pix/deg) = pix radius
                # print(xpix,ypix,rpix) # debug: print file values
                # crop image with matplotlib
                plt.imshow(image[int((ypix-rpix)):int((ypix+rpix)),int((xpix-rpix)):int((xpix+rpix))],cmap='gray',vmin=0,vmax=1.5,origin='lower')
                plt.title(f'f770w_obj{obj['Name'][i]}')
                plt.savefig(f'output/obj{obj['Name'][i]}.png')
                f.close(output_verify='silentfix') # close up shop
                break # if we find it, we can leave.
            else:
                print('Absent.') # it aint here
                f.close(output_verify='silentfix') # close up shop
        else:
            print('Absent.') # it aint here
            f.close(output_verify='silentfix') # close up shop
print("Finished" + "!"*100)

##### Range File

Bonus: Lets find the ranges for all of the fits.

In [10]:
from os import listdir
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
import numpy as np
import multiprocessing as mp
path = r"/home/eliasw/OneDrive/Research/Deconvolution/fits/miri"

import warnings
warnings.filterwarnings("ignore")

Importing our cool functions we made.

In [2]:
# Function for parallel RA/Dec conversion
def convert_pixels_to_radec(pixel_chunk, wcs):
    return w.all_pix2world(pixel_chunk, 1)

In [4]:
def bounds(image,w):
    image_shape = image.shape  # Assuming 2D image

    # Generate a grid of pixel coordinates for the entire image
    y_pixels, x_pixels = np.indices(image_shape)

    # Define subsampling factors for different regions
    edge_subsample_factor = 10   # Dense sampling for edge
    middle_subsample_factor = 1000  # Sparse sampling for the middle

    # Define the boundaries for edges and middle regions
    # Shrink the edge region, so more of the image is middle
    edge_boundary = (image_shape[0] // 10, image_shape[1] // 10)  # Shrinking the edge (12.5% from edges)
    middle_boundary = (image_shape[0] // 4, image_shape[1] // 4)  # Expanding the middle region (25% from edges)

    # Generate a grid of pixel coordinates for the entire image
    y_pixels, x_pixels = np.indices(image_shape)

    # Sample the edges (outside the shrunk middle region)
    x_pixels_edges = np.concatenate([x_pixels[::edge_subsample_factor, 0],  # Left edge
                                    x_pixels[::edge_subsample_factor, -1],  # Right edge
                                    x_pixels[0, ::edge_subsample_factor],  # Top edge
                                    x_pixels[-1, ::edge_subsample_factor]])  # Bottom edge
    y_pixels_edges = np.concatenate([y_pixels[::edge_subsample_factor, 0], 
                                    y_pixels[::edge_subsample_factor, -1],
                                    y_pixels[0, ::edge_subsample_factor],
                                    y_pixels[-1, ::edge_subsample_factor]])

    # Sample the middle region (less dense than center but more dense than edges)
    x_pixels_middle = x_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                            edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]
    y_pixels_middle = y_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                            edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]


    # Combine all sampled pixels
    pixels_combined = np.column_stack([np.concatenate([x_pixels_edges.ravel(), x_pixels_middle.ravel()]),
                                    np.concatenate([y_pixels_edges.ravel(), y_pixels_middle.ravel()])])

    # Split the pixel blocks into chunks for parallel processing
    num_cores = mp.cpu_count()
    pixel_chunks = np.array_split(pixels_combined, num_cores)

    # Convert pixel chunks to RA/Dec in parallel
    with mp.Pool(num_cores) as pool:
        ra_dec_chunks = pool.starmap(convert_pixels_to_radec, [(chunk, w) for chunk in pixel_chunks])

    # Combine the RA/Dec results
    ra_dec = np.concatenate(ra_dec_chunks, axis=0)

    # Extract Right Acension (RA) values and find the min and max
    ra_values = ra_dec[:, 0]  # RA is the first column (in degrees)
    ramin = np.min(ra_values)
    ramax = np.max(ra_values)

    # Extract Declination (Dec) values and find the min and max
    dec_values = ra_dec[:, 1]  # Dec is the second column (in degrees)
    decmin = np.min(dec_values)
    decmax = np.max(dec_values)

    return ramin,ramax,decmin,decmax

In [11]:
# prepare file stuff
l = open('fits/fitsrange.txt', 'w')
l.write('File\tRAmin\tRAmax\tDECmin\tDECmax\n')
for file in listdir(path):
    if file.endswith('.fits.gz'):
        f = fits.open(path + "/" + file)
        w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
        image = f[1].data
        # get coord bounds
        ramin,ramax,decmin,decmax = bounds(image,w)
        l.write(f'{file}\t{ramin}\t{ramax}\t{decmin}\t{decmax}\n')

Now let's look at what we got.

In [12]:
range = pd.read_csv('fits/fitsrange.txt',sep='\t',header=0)

In [14]:
range

,File,RAmin,RAmax,DECmin,DECmax


In [29]:
print(f'Total fits range: RA:{min(range['RAmin'])} - {max(range['RAmax'])}\tDEC:{min(range['DECmin'])} - {max(range['DECmax'])}')

Total fits range: RA:149.65411536132606 - 150.48360984203632	DEC:1.7076893037604302 - 2.3353418126877874


Now we have a range of coordinates where we probably have galaxies.\
Our view composes of a rectangle with a center of 150.06886, 2.0215156 (RA/DEC), with sides of 0.6880611 RA and 0.39394768 DEC. (deg)\
We could use this to make an upper bound on the view of the galaxies we want.

##### Actual Data

Now we are gonna test it on actual data. \
COSMOS2020 has 1.7 million galaxies in the general catalog. I don't think we can run that. We are instead going to try to find the top 100 galaxies of the catalog. \
They're around the area of our fits files.

In [2]:
from os import listdir
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
import numpy as np
import multiprocessing as mp
path = r"/home/eliasw/OneDrive/Research/Deconvolution/fits"

import warnings
warnings.filterwarnings("ignore")

Importing our cool functions we made.

In [2]:
# Function for parallel RA/Dec conversion
def convert_pixels_to_radec(pixel_chunk, wcs):
    return w.all_pix2world(pixel_chunk, 1)

In [3]:
def bounds(image,w):
    image_shape = image.shape  # Assuming 2D image

    # Generate a grid of pixel coordinates for the entire image
    y_pixels, x_pixels = np.indices(image_shape)

    # Define subsampling factors for different regions
    edge_subsample_factor = 10   # Dense sampling for edge
    middle_subsample_factor = 1000  # Sparse sampling for the middle

    # Define the boundaries for edges and middle regions
    # Shrink the edge region, so more of the image is middle
    edge_boundary = (image_shape[0] // 10, image_shape[1] // 10)  # Shrinking the edge (12.5% from edges)
    middle_boundary = (image_shape[0] // 4, image_shape[1] // 4)  # Expanding the middle region (25% from edges)

    # Generate a grid of pixel coordinates for the entire image
    y_pixels, x_pixels = np.indices(image_shape)

    # Sample the edges (outside the shrunk middle region)
    x_pixels_edges = np.concatenate([x_pixels[::edge_subsample_factor, 0],  # Left edge
                                    x_pixels[::edge_subsample_factor, -1],  # Right edge
                                    x_pixels[0, ::edge_subsample_factor],  # Top edge
                                    x_pixels[-1, ::edge_subsample_factor]])  # Bottom edge
    y_pixels_edges = np.concatenate([y_pixels[::edge_subsample_factor, 0], 
                                    y_pixels[::edge_subsample_factor, -1],
                                    y_pixels[0, ::edge_subsample_factor],
                                    y_pixels[-1, ::edge_subsample_factor]])

    # Sample the middle region (less dense than center but more dense than edges)
    x_pixels_middle = x_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                            edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]
    y_pixels_middle = y_pixels[edge_boundary[0]:-edge_boundary[0]:middle_subsample_factor,
                            edge_boundary[1]:-edge_boundary[1]:middle_subsample_factor]


    # Combine all sampled pixels
    pixels_combined = np.column_stack([np.concatenate([x_pixels_edges.ravel(), x_pixels_middle.ravel()]),
                                    np.concatenate([y_pixels_edges.ravel(), y_pixels_middle.ravel()])])

    # Split the pixel blocks into chunks for parallel processing
    num_cores = mp.cpu_count()
    pixel_chunks = np.array_split(pixels_combined, num_cores)

    # Convert pixel chunks to RA/Dec in parallel
    with mp.Pool(num_cores) as pool:
        ra_dec_chunks = pool.starmap(convert_pixels_to_radec, [(chunk, w) for chunk in pixel_chunks])

    # Combine the RA/Dec results
    ra_dec = np.concatenate(ra_dec_chunks, axis=0)

    # Extract Right Acension (RA) values and find the min and max
    ra_values = ra_dec[:, 0]  # RA is the first column (in degrees)
    ramin = np.min(ra_values)
    ramax = np.max(ra_values)

    # Extract Declination (Dec) values and find the min and max
    dec_values = ra_dec[:, 1]  # Dec is the second column (in degrees)
    decmin = np.min(dec_values)
    decmax = np.max(dec_values)

    return ramin,ramax,decmin,decmax

Let's get those galaxies.

In [4]:
obj = pd.read_csv('cosmos2020classic100.csv',sep=',',header=0)

In [5]:
obj

,Classic,RAJ2000,DEJ2000,COSMOS2015
0,536,150.233240,1.392387,NaN
1,1025,150.234091,1.392218,NaN
2,1026,150.233957,1.392599,NaN
3,72,150.231391,1.391675,NaN
4,455,150.230869,1.392225,NaN
...,...,...,...,...
96,2829,150.219695,1.395190,NaN
97,944,150.214110,1.392984,NaN
98,1006,150.212793,1.393061,NaN
99,1955,150.214005,1.394012,NaN


First, we are going to have it tell us if, and how many, galaxies are in there, without printing picures. Then it will be picture time.

In [ ]:
foundgals = 0
for i in range(len(obj)):
    print(f'Looking for Object {obj['Classic'][i]}..\t ({i}/{len(obj)})') # Debug: What object are we on?
    for file in listdir(path): # look through each file
        print(file) # debug: print file name
        f = fits.open(path + "/" + file)
        w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
        image = f[1].data
        # get coord bounds
        ramin,ramax,decmin,decmax = bounds(image,w)
        xpix, ypix = w.all_world2pix(obj['RAJ2000'][i],obj['DEJ2000'][i],1) # convert from RA/DEC to pixel position
        if (0 <= xpix <= f[1].header['NAXIS1'] and 0 <= ypix <= f[1].header['NAXIS2']):
            if ((ramin<=obj['RAJ2000'][i]<=ramax) & (decmin<=obj['DEJ2000'][i]<=decmax)) & (f[4].data[int(ypix),int(xpix)] != 0): # if our object's values are between the files values
                print(f'Object {obj['Classic'][i]} is in {file}')
                foundgals += 1
                f.close() # close up shop
                break # if we find it, we can leave.
            else:
                print('Absent.') # it aint here
                f.close() # close up shop
        else:
            print('Absent.') # it aint here
            f.close() # close up shop
    print(f'Object {obj['Classic'][i]} is not in our fits.')
print("Finished" + "!"*100)
print(f'Found {foundgals} galaxies out of {len(obj)}')

Damn. No galaxies. Probably should throw a test galaxy in there, that we know is in our fits files.\
Done. 'Spiral Dude' is our test, and IS in our range, so if it says he isn't, there's a slight problem.

Let's also take our 100 (now 150) galaxies from a section within our range, which we found above.\
" Total fits range: RA:149.65411536132606 - 150.48360984203632	DEC:1.7076893037604302 - 2.3353418126877874 "

Now we check if it works.

In [5]:
obj = pd.read_csv('withinrangeCOSMOS2020.csv',sep=',',header=0)

In [6]:
obj

,Classic,RAJ2000,DEJ2000
0,419178,149.725114,1.821626
1,418093,149.720245,1.821483
2,418141,149.720799,1.821483
3,418229,149.721477,1.822020
4,418311,149.722271,1.822107
...,...,...,...
159246,840560,149.984543,2.220890
159247,840952,149.986198,2.220803
159248,842058,149.984968,2.221499
159249,840409,149.964419,2.220981


That's a lot. But less than before.

In [ ]:
foundgals = 0
for i in range(len(obj)):
    print(f'Looking for Object {obj['Classic'][i]}..\t ({i+1}/{len(obj)-1})') # Debug: What object are we on?
    for file in listdir(path): # look through each file
        print(file) # debug: print file name
        f = fits.open(path + "/" + file)
        w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
        image = f[1].data
        # get coord bounds
        ramin,ramax,decmin,decmax = bounds(image,w)
        xpix, ypix = w.all_world2pix(obj['RAJ2000'][i],obj['DEJ2000'][i],1) # convert from RA/DEC to pixel position
        if (0 <= xpix <= f[1].header['NAXIS1'] and 0 <= ypix <= f[1].header['NAXIS2']):
            if ((ramin<=obj['RAJ2000'][i]<=ramax) & (decmin<=obj['DEJ2000'][i]<=decmax)) & (f[4].data[int(ypix),int(xpix)] != 0): # if our object's values are between the files values
                print(f'Object {obj['Classic'][i]} is in {file}')
                foundgals += 1
                f.close() # close up shop
                break # if we find it, we can leave.
            else:
                print('Absent.') # it aint here
                f.close() # close up shop
                break
        else:
            print('Absent.') # it aint here
            f.close() # close up shop
print("Finished" + "!"*100)
print(f'Found {foundgals} galaxies out of {len(obj)}')

14 objects in 9 minutes

Nice! It works! ... But it's really really slow. \
Maybe we could just use our txt file and not calculate the ranges each time?

In [7]:
ranges = pd.read_csv('fitsrange.txt',sep='\t',header=0)

In [8]:
ranges

,File,RAmin,RAmax,DECmin,DECmax
0,mosaic_miri_f770w_COSMOS-Web_60mas_A4_v0_5_i2d...,150.126579,150.348060,1.937735,2.187454
1,mosaic_miri_f770w_COSMOS-Web_60mas_A5_v0_5_i2d...,150.262128,150.483610,1.888411,2.138133
2,mosaic_miri_f770w_COSMOS-Web_60mas_A10_v0_5_i2...,150.196296,150.417747,1.707689,1.957401
3,example.fits.gz,149.978850,150.025232,1.994487,2.049157
4,mosaic_miri_f770w_COSMOS-Web_60mas_A1_v0_5_i2d...,149.719886,149.941346,2.085646,2.335342
5,mosaic_miri_f770w_COSMOS-Web_60mas_A3_v0_5_i2d...,149.991022,150.212498,1.987050,2.236763
6,mosaic_miri_f770w_COSMOS-Web_60mas_A2_v0_5_i2d...,149.855457,150.076926,2.036353,2.286059
7,mosaic_miri_f770w_COSMOS-Web_60mas_A8_v0_5_i2d...,149.925220,150.146662,1.806313,2.056018
8,mosaic_miri_f770w_COSMOS-Web_60mas_A7_v0_5_i2d...,149.789670,150.011104,1.855610,2.105309
9,mosaic_miri_f770w_COSMOS-Web_60mas_A6_v0_5_i2d...,149.654115,149.875539,1.904897,2.154587


In [ ]:
print(float(ranges[ranges['File'] == file]['RAmin'])) # gets the ramin value for the column with a filename that matches the filename

Optimizations:
* Instead of calculating range of .fits files, instead pull from a txt file of ranges calculated beforehand.
* Reorder If and Loops so that resourse intensive operations are only used when they are needed. If it fails, it doesn't have to run those.
* Introduce multiprocessing to allow use of all cores on the pc.

NOTE: IF ALL FITS FILES HAVE SAME AXIS LENGHTS NAXIS1 AND 2 then JUST PUT THOSE IN, ELSE, KEEP CODE SAME

In [ ]:
foundgals = 0
for i in range(len(obj)):
    print(f'Looking for Object {obj['Classic'][i]}..\t ({i+1}/{len(obj)-1})') # Debug: What object are we on?
    for file in listdir(path): # look through each file
        # print(file) # debug: print file name
        # get coord bounds
        ramin = float(ranges[ranges['File'] == file]['RAmin'])
        ramax = float(ranges[ranges['File'] == file]['RAmax'])
        decmin = float(ranges[ranges['File'] == file]['DECmin'])
        decmax = float(ranges[ranges['File'] == file]['DECmax'])
        if (ramin <= obj['RAJ2000'][i] <= ramax) and (decmin <= obj['DEJ2000'][i] <= decmax):
            #print('in range')
            f = fits.open(path + "/" + file)
            w = WCS(f[1].header,fix=False) # sets up world coordinate system conversions; fix=false fixes it spitting a bunch of wordvomit as it fixes all of the nonstandard stuff in the fits. i dont think it changes the result. if it does. uncomment.
            xpix, ypix = w.all_world2pix(obj['RAJ2000'][i],obj['DEJ2000'][i],1) # convert from RA/DEC to pixel position
            if (0 <= xpix <= f[1].header['NAXIS1'] and 0 <= ypix <= f[1].header['NAXIS2']):
                #print(f'0 <= {xpix} <= {f[1].header['NAXIS1']} and 0 <= {ypix} <= {f[1].header['NAXIS2']}) <TRUE>')
                if (f[4].data[int(ypix),int(xpix)] != 0):
                    #print(f'{f[4].data[int(ypix),int(xpix)]} != 0')
                    print(f'Object {obj['Classic'][i]} is in {file}')
                    foundgals += 1
                    f.close() # close up shop
                    break # if we find it, we can leave.
                else:
                    #print(f'({f[4].data[int(ypix),int(xpix)]} != 0) <FALSE>')
                    f.close()
                    continue
            else:
                #print('Absent.') # it aint here
                #print(f'0 <= {xpix} <= {f[1].header['NAXIS1']} and 0 <= {ypix} <= {f[1].header['NAXIS2']}) <FALSE>')
                f.close() # close up shop
                continue
print("Finished" + "!"*100)

Preloading lets us keep the files in memory, so we don't have to read it in, but it takes some RAM. Crashes if we read in the weight files.

In [7]:
path = r"/home/#######/Research/Deconvolution/fits"
# Preload FITS headers and data
def preload_fits_files(pa):
    fits_data = {}
    for file in listdir(pa):
        if file.endswith('.fits'):  # Ensure it's a FITS file
            f = fits.open(path + "/" + file)
            header = f[1].header # Get WCS header for pixel to RA/Dec conversion
            weight = f[4].data  # Load only the necessary data extension
            fits_data[file] = {'wcs': header, 'weight': weight}
            f.close()  # Close the file after loading the necessary information
    return fits_data


We are making masks now

If we make a binary mask of where we have data, we can use it with less space.

In [1]:
from os import listdir
from astropy.io import fits
import numpy as np
path = r"/home/eliasw/OneDrive/Research/Deconvolution/fits/miri"
import warnings
warnings.filterwarnings("ignore")

for file in listdir(path): # look through each file
    if file.endswith('.fits.gz'):  # Ensure it's a FITS file
        # Load the weight array
        weight = fits.getdata(path + "/" + file, ext=4)
        # Initialize immask as an array of zeros with the same shape as weight
        immask = np.zeros_like(weight, dtype=int)
        # Set immask to 1 where weight is greater than 0
        immask[weight > 0] = 1
        # Convert immask to an 8-bit integer
        immask_8bit = immask.astype(np.uint8)
        # Open the original FITS file to copy the header from the weight extension
        header = fits.getheader(path + "/" + file, ext=4)  # Assuming ext=4 for weight
        header['EXTNAME'] = "MASK"
        # Create a new FITS PrimaryHDU with immask data and header
        hdu = fits.PrimaryHDU(data=immask_8bit, header=header)
        # Save the new FITS file with the mask data
        output_path = path + "/mask/" + file
        output_path = output_path.replace('.fits.gz','_mask.fits')
        hdu.writeto(output_path, overwrite=True)
        print(f"Mask FITS file saved at {output_path}")

Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A4_v0_5_i2d_mask.fits
Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A5_v0_5_i2d_mask.fits
Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A10_v0_5_i2d_mask.fits
Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A1_v0_5_i2d_mask.fits
Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A3_v0_5_i2d_mask.fits
Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A2_v0_5_i2d_mask.fits
Mask FITS file saved at /home/eliasw/OneDrive/Research/Deconvolution/fits/miri/mask/mosaic_miri_f770w_COSMOS-Web_60mas_A8_v0_5_i2d_mask.fits
Mask FITS fi

In [7]:
def preload_fits_files(path):
    fits_data = {}
    for file in listdir(path):
        if file.endswith('.fits.gz'):  # Ensure it's a FITS file
            header = WCS(fits.getheader(path + '/' + file, ext=1), fix=False)  # Get WCS header for pixel to RA/Dec conversion
            mask = fits.getdata((path + '/mask/' + file).replace('.fits.gz','_mask.fits'))  # Load only the necessary data extension
            fits_data[file] = {'wcs': header, 'mask': mask}
    return fits_data

THIS WORKS SO MUCH BETTER.\
DID ALL 160k in 5 minutes.

### Final Version (also in the form of a .py function)

In [ ]:
# import dependancies
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from os import listdir
import multiprocessing as mp

# focus out warnings about fits handling that bloat the console
import warnings
warnings.filterwarnings("ignore")

# Define the path to the FITS files
path = r"/home/#/Research/Deconvolution/fits"

# Load the galaxy data and FITS file ranges
obj = pd.read_csv('withinrangeCOSMOS2020.csv', sep=',', header=0)
ranges = pd.read_csv('fitsrange.txt', sep='\t', header=0)

# Preload FITS headers and data
def preload_fits_files(path):
    fits_data = {}
    for file in listdir(path):
        if file.endswith('.fits.gz'):  # Ensure it's a FITS file
            header = WCS(fits.getheader(path + '/' + file, ext=1), fix=False)  # Get WCS header for pixel to RA/Dec conversion
            mask = fits.getdata((path + '/mask/' + file).replace('.fits.gz','_mask.fits'))  # Load only the necessary data extension
            fits_data[file] = {'wcs': header, 'mask': mask}
    return fits_data

# Function to search for galaxies in preloaded FITS files for a subset of galaxies
def search_galaxies(start_index, end_index, fits_data):
    foundgals = 0
    found_objects = []
    end_index = min(end_index, len(obj)) #indexing fix
    #print(f"Processing indices from {start_index} to {end_index}")

    for i in range(start_index, end_index):
       # print(f"Checking index {i}, Galaxy ID {obj['Classic'][i]}") 
        for file, file_data in fits_data.items():  # Loop through preloaded FITS data
            ramin = float(ranges[ranges['File'] == file]['RAmin'])
            ramax = float(ranges[ranges['File'] == file]['RAmax'])
            decmin = float(ranges[ranges['File'] == file]['DECmin'])
            decmax = float(ranges[ranges['File'] == file]['DECmax'])
            # Check if the galaxy is within the RA/Dec range of the file
            if ramin <= obj['RAJ2000'][i]:
                if obj['RAJ2000'][i] <= ramax:
                    if decmin <= obj['DEJ2000'][i]:
                        if obj['DEJ2000'][i] <= decmax:
                            w = file_data['wcs']  # Get the preloaded WCS header
                            xpix, ypix = w.all_world2pix(obj['RAJ2000'][i], obj['DEJ2000'][i], 1)
                            # Check if the pixel coordinates are within bounds
                            if (0 <= xpix <= 9600 and 0 <= ypix <= 12455):
                                if (file_data['mask'][int(ypix),int(xpix)] == True):  # Galaxy found
                                    foundgals += 1
                                    #print(f"Galaxy found at index {i}, ID {obj['Classic'][i]} in file {file}, at coords {obj['RAJ2000'][i]} RA and {obj['DEJ2000'][i]} DEC")
                                    found_objects.append([obj['Classic'][i], obj['RAJ2000'][i], obj['DEJ2000'][i], file])  # Save RA, Dec, and file
                                    break  # Stop searching this file if galaxy found
    
    return foundgals, found_objects

# Multi-core processing: split the galaxy list into chunks
if __name__ == '__main__':
    num_cores = mp.cpu_count()  # Use all available cores
    chunk_size = len(obj) // num_cores  # Determine the size of each chunk

    # Preload all FITS files and headers before starting the search
    fits_data = preload_fits_files(path)

    # Create a pool of workers and distribute the work
    with mp.Pool(num_cores) as pool:
        results = [pool.apply_async(search_galaxies, args=(i, i + chunk_size, fits_data)) for i in range(0, len(obj), chunk_size)]
        
        found_gals = 0
        found_objects = []

        for result in results:
            chunk_found_gals, chunk_found_details = result.get()  # Get both elements from the result tuple
            found_gals += chunk_found_gals  # Sum the galaxy count
            found_objects.extend(chunk_found_details)  # Add the found details to the main list

        # Write the results to a file
        with open('found_galaxies.txt', 'w') as out:
            out.write(f'Found {found_gals} galaxies out of {len(obj)}\n')
            out.write('ID\tRAJ2000\tDEJ2000\tFile\n')  # Header for the galaxy details
            for gal in found_objects:
                out.write(f'{gal[0]}\t{gal[1]}\t{gal[2]}\t{gal[3]}\n')  # Write each galaxy's details (RA, Dec, and file)
            out.flush()